In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LSTM, Embedding, Dense, Input
from tensorflow.keras.optimizers import RMSprop, Adadelta
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import utils
from tensorflow.keras.utils import plot_model

import yaml

In [2]:
path = '/Users/jokkojja/Downloads/ijcnlp_dailydialog/dialogues_text.txt'

# Preparing data

In [4]:
with open(path, 'r') as file:
    document = file.read()

In [15]:
document = document.replace('__eou__', '').split('\n')

In [28]:
test_doc = document[:10]
document = [sent.split('?') for sent in document] # naive approach

In [41]:
# question-answer tags

questions = []
answers = []

# process dialog

for con in document:
#     print(con, '\n')
    if len(con) > 2:
        questions.append(con[0])
        replies = con[1:]
        ans = ''
        for rep in replies:
            ans+= " " + rep
            answers.append(ans)
    elif len(con) == 2:
        questions.append(con[0])
        answers.append(con[1])
cleaned_answers = []

for i, ans in enumerate(answers):
    if isinstance(ans, str):
        cleaned_answers.append(ans)
    else:
        questions.pop(i)
        
#tags
answers = []
for i, ans in enumerate(cleaned_answers):
    answers.append('<START>' + ans + '<END>')


In [42]:
print(f'Question: {questions[150]}, Answer: {answers[150]}')

Question: Peter , wash your hands first , and then have some dessert .  Mom , can I have some ice cream , Answer: <START>   Tom has gone to lunch .  I'll just try him again .  Do you want me to have him give you a call    No , but what time do you expect him back    He should be back in about forty minutes . <END>


In [53]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)
vocab = list(tokenizer.word_index.items())
print(f'Vocab len: {len(vocab)}')

Vocab len: 18750


## Sample preparation

In [ ]:
tokenized_questions = tokenizer.texts_to_sequences(questions)
max_question_len =max([len(x) for x in tokenized_questions])
